In [1]:
from konlpy.tag import Okt

In [2]:
okt = Okt()

In [3]:
def build_bag_of_words(document):
    document = document.replace(".", "")
    tokenized_document = okt.morphs(document)

    word_to_index = {}
    bow = []

    for word in tokenized_document:
        if word not in word_to_index.keys():
            word_to_index[word] = len(word_to_index)
            bow.insert(len(word_to_index) - 1, 1)
        else:
            index = word_to_index[word]
            bow[index] += 1

    return word_to_index, bow

In [5]:
doc1 = "정부가 발표하는 물가상승률과 소비자가 느끼는 물가상승률은 다르다."
vocab, bow = build_bag_of_words(doc1)
vocab, bow

({'정부': 0,
  '가': 1,
  '발표': 2,
  '하는': 3,
  '물가상승률': 4,
  '과': 5,
  '소비자': 6,
  '느끼는': 7,
  '은': 8,
  '다르다': 9},
 [1, 2, 1, 1, 2, 1, 1, 1, 1, 1])

In [7]:
doc2 = "소비자는 주로 소비하는 상품을 기준으로 물가상승률을 느낀다."
vocab, bow = build_bag_of_words(doc2)
vocab, bow

({'소비자': 0,
  '는': 1,
  '주로': 2,
  '소비': 3,
  '하는': 4,
  '상품': 5,
  '을': 6,
  '기준': 7,
  '으로': 8,
  '물가상승률': 9,
  '느낀다': 10},
 [1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1])

In [8]:
doc3 = doc1 + " " + doc2
vocab, bow = build_bag_of_words(doc3)
vocab, bow

({'정부': 0,
  '가': 1,
  '발표': 2,
  '하는': 3,
  '물가상승률': 4,
  '과': 5,
  '소비자': 6,
  '느끼는': 7,
  '은': 8,
  '다르다': 9,
  '는': 10,
  '주로': 11,
  '소비': 12,
  '상품': 13,
  '을': 14,
  '기준': 15,
  '으로': 16,
  '느낀다': 17},
 [1, 2, 1, 2, 3, 1, 2, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1])

BoW는 종종 여러 문서의 단어 집합을 합친 뒤, 해당 단어 집합에 대한 각 문서의 BoW를 구하기도 한다. 가령, 문서 3에 대한 단어 집합을 기준으로 문서 1, 2의 BoW를 만들면 결과는 아래와 같다.

문서3 단어 집합에 대한 문서1 BoW : [1, 2, 1, 1, 2, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0,
0, 0, 0]<br>
문서3 단어 집합에 대한 문서2 BoW : [0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 2,
1, 1, 1]

BoW는 주로 어떤 단어가 얼마나 등장했는지를 기준으로 문서가 어떤 성격을 가지는지 판단할 때 쓰인다.

scikit-learn에서 단어의 빈도를 Count하여 Vector로 만드는 CountVectorizer 클래스를 지원한다. 이를 이용하면 영어에 대해 손쉽게 BoW를 만들 수 있다.

In [9]:
from sklearn.feature_extraction.text import CountVectorizer

In [15]:
corpus = ["you know I want your love. because I love you."]
vector = CountVectorizer()

print(f"bow: {vector.fit_transform(corpus).toarray()}\nvocab: {vector.vocabulary_}")

bow: [[1 1 2 1 2 1]]
vocab: {'you': 4, 'know': 1, 'want': 3, 'your': 5, 'love': 2, 'because': 0}


결과에서 보면 알 수 있듯이 CountVectorizer는 길이가 2 이상인 문자에 대해서만 토큰으로 인식하므로 I가 사라진 것을 알 수 있다.

BoW는 띄어쓰기 기준으로만 분리하므로 영어에서는 별 문제가 없지만 한국어에서는 제대로 된 BoW가 만들어질 수 없다.

CountVectorizer에 불용어를 지정하면 불용어를 제외하고 BoW를 만들 수 있다.

In [16]:
text = ["Family is not an important thing. It's everything."]
# 사용자 직접 정의 불용어 사용
vect = CountVectorizer(stop_words=["the", "a", "an", "is", "not"])
print(f"bow: {vect.fit_transform(text).toarray()}\nvocab: {vect.vocabulary_}")

bow: [[1 1 1 1 1]]
vocab: {'family': 1, 'important': 2, 'thing': 4, 'it': 3, 'everything': 0}


In [18]:
# CountVectorizer 제공 자체 불용어 사용
vect = CountVectorizer(stop_words="english")
print(f"bow: {vect.fit_transform(text).toarray()}\nvocab: {vect.vocabulary_}")

bow: [[1 1 1]]
vocab: {'family': 0, 'important': 1, 'thing': 2}


In [20]:
from nltk.corpus import stopwords

stop_words = stopwords.words("english")
vect = CountVectorizer(stop_words=stop_words)
print(f"bow: {vect.fit_transform(text).toarray()}\nvocab: {vect.vocabulary_}")

bow: [[1 1 1 1]]
vocab: {'family': 1, 'important': 2, 'thing': 3, 'everything': 0}
